# Transformer
Bienvenue à l'un des TPs les plus longues de ce repo. Aujourd'hui on va s'intéresser à l'architecture des transformers.
Un transformer est un réseau de neurones qui permet de transformer des séquences de données en d'autres séquences de données. Il est très utilisé dans le domaine du NLP pour faire de la traduction de texte par exemple. C'est aussi l'architecture derrière le fameux ChatGPT, que vous connaissez déjà.

Les modèles de génération d'images comme DALLE utilisent aussi des transformers, pour encoder le texte.

![chatgpt](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/chatgpt.png)

![dalle](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/dalle.png)

## Overview

Personnellement, je trouve que c'est plus facile à comprendre les transformers en commençant par comprendre comment l'utiliser (comprendre les entrées et les sorties) et ensuite en regardant l'architecture.
On va donc commencer par un exemple d'utilisation, puis on va rentrer dans le détail de l'architecture.
Donc pour l'instant, on va considérer le transformer comme un black box.


## Inférence (ou la phase d'utilisation/prédiction)

Pendant l'inférence, le transformer prédit un token (ou un mot, ou un élément de la séquence, c'est la même chose) à la fois. Il prend en compte les tokens précédemment prédits et la séquence d'origine comme entrées pour prédire le token suivant.

Considérons l'exemple de la traduction : le transformer utilise à la fois la phrase source et les mots précédemment prédits de la phrase traduite comme entrées afin de générer le mot suivant de la phrase traduite.

![blackbox](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/blackbox.png)

Pendant la traduction, le transformer prend en entrée la phrase source et la phrase actuellement prédite, puis génère le mot suivant dans la phrase traduite en se basant sur ces deux entrées. Il répète ce processus, en prenant les mots précédemment prédits et la phrase source en entrée à chaque étape, jusqu'à ce que la phrase traduite complète soit générée.

Examinons ça avec un exemple détaillé.

Supposons que le transformer soit déjà entraîné et qu'on veut qu'il traduise "I love oranges." en français. La phrase traduite qu'on veut obtenir est "J'aime les oranges".

La phrase source est `["I", "love", "oranges", "."]`.

Comme c'est le début de l'inférence, la phrase actuellement prédite est vide, mais on ne peut pas mettre une phrase vide dans le transformer, donc on commence par `["<start>"]` comme phrase actuellement prédite (target sequence).

On a donc ces deux séquences en entrées: `["I", "love", "oranges", "."]` et `["<start>"]`.

On va tokenizer ensuite ces séquences (convertir chaque mot en un nombre le représentant), par exemple `["I", "love", "oranges", "."]` -> `[10, 35, 20, 49]` et `["<start>"]` -> `[40]`.

Le transformer prend ensuite en entrée ces deux séquences, puis génère un vecteur de probabilité sur l'ensemble du vocabulaire, représentant la probabilité de chaque mot possible suivant le token `<start>`.

Puisque le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être `J'`. On ajoute `J'` à la phrase actuellement prédite.

![first inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability1.png)

On a donc maintenant `["I", "love", "oranges", "."]` et `["<start>", "J'"]`. Avec la tokenisation, on a `[10, 35, 20, 49]` et `[40, 20]`, en supposant que 20 représente `J'`.
Remarque qu'on a deux tokeniseurs différents pour l'anglais et le français.

Le transformer prend ensuite en entrée ces séquences, puis génère deux vecteurs de probabilité, la première représentant la probabilité du mot possible suivant le token `<start>`, et la seconde celle du mot possible suivant le token `J'`.

![second inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability2.png)

Remarque que le premier vecteur de probabilité est identique au précédent. C'est parce que le transformer génère toujours les même vecteurs pour les mêmes entrées même s'il y a des tokens supplémentaires dans la phrase actuellement prédite. Les futurs tokens n'influencent pas les vecteurs de probabilité des tokens précédents. C'est parce qu'il y a un masque qu'on appelle `causal mask` qui empêche les tokens de prêter attention aux futurs tokens; on en discutera plus tard dans la section sur l'architecture du transformer.

Puisque on a déjà prédit le mot suivant le token `<start>` à la première étape, on peut ignorer le premier vecteur de probabilité et se concentrer uniquement sur la seconde.


Comme le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être "aime".

On obtient donc `["I", "love", "oranges", "."]` et `["<start>", "J'", "aime"]`.

On répète ce processus jusqu'à ce que le transformer génère le token `<end>`, indiquant que la séquence générée est complète. Le transformer est entraîneé pour prédire le token <end> de manière appropriée pendant la phase d'entraînement, ce qui nous permet de déterminer quand arrêter la boucle de génération.

### Exercice 1: Utilisation d'un transformer pré-entraiîné pour faire de la traduction de texte
On va importer un modèle de transformer déjà entraîné et vous allez devoir faire la boucle d'inférence pour traduire la phrase "I love oranges." en français.

Téléchargement des libraries: `pip install transformers sentencepiece`.
Si vous êtes sur colab, alors lancez cette cellule

In [ ]:
!pip install transformers sentencepiece

Maintenant on va load le modèle. Vous n'avez pas besoin de comprendre la cellule suivante, tout ce qu'il faut retenir c'est que vous avez un objet `transformer` qui contient le modèle et le tokenizer que vous allez utiliser pour faire l'inférence.

Je vais vous expliquer comment utiliser le modèle et le tokenizer juste après.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch

class MyTransformer:
    def __init__(self):
        # Load pre-trained model and tokenizer
        model_name = "Helsinki-NLP/opus-mt-en-fr"
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)
        self.start_token = self.model.config.decoder_start_token_id

    def tokenize(self, text):
        tokenized_input = self.tokenizer(text, return_tensors='pt', truncation=True)
        return tokenized_input["input_ids"]

    def detokenize(self, token_ids):
        return self.tokenizer.batch_decode(token_ids, skip_special_tokens=True)

    def forward(self, src_sequence, tgt_sequence):
        """
        src_sequence: torch.LongTensor of shape (batch_size, seq_len) la séquence des ids des mots de la phrase source
        tgt_sequence: torch.LongTensor of shape (batch_size, seq_len) la séquence des ids des mots de la phrase actuellement prédite

        return: torch.FloatTensor of shape (batch_size, seq_len, vocab_size) la probabilité de chaque mot possible pour chaque token de la phrase actuellement prédite
        """
        with torch.no_grad():
            encoder_states = self.model.get_encoder()(src_sequence)
            decoder_states = self.model.get_decoder()(tgt_sequence, encoder_hidden_states=encoder_states.last_hidden_state)
            prediction = self.model.lm_head(decoder_states.last_hidden_state)
            prediction = torch.softmax(prediction, dim=-1)
        return prediction

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

transformer = MyTransformer()

Dans `transformer`, vous avez 2 méthodes que vous pouvez utiliser pour l'inférence: `transformer.tokenize` et `transformer.detokenize` pour convertir les phrases en nombres et vice versa.

Lorsque vous faites ```transformer.tokenize("I love oranges.")```, vous obtenez ```tensor([[47, 1779, 12610, 9, 3, 0]])```,

Et lorsque vous faites ```transformer.detokenize([47, 1779, 12610, 9, 3, 0])```, vous obtenez ```['I', 'love', 'orange', 's', '.', '</s>']```.

`</s>` est le token `<end>`.

On remarque aussi que le tokenizer sépare le mot "orange" en deux tokens, "orange" et "s". C'est juste une façon de faire de la tokenisation, et il y a d'autres façons de faire comme du mot par mot ou du caractère par caractère.

Pour utiliser le modèle pré entraîné, vous allez devoir faire `transformer(input_ids, decoder_input_ids)`, où `input_ids` est la séquence tokenisée de la phrase source et `decoder_input_ids` est la séquence tokenisée de la phrase actuellement prédite. `input_ids` et `decoder_input_ids` doivent être des `torch.tensor` de shape `(batch_size, seq_len)` et de type `long`.

Par exemple `transformer.tokenize("I love oranges.")` est un valide `input_ids` car il est de shape `(1, 6)` et de type `long`.

`transformer(input_ids, decoder_input_ids)` va retourner un `torch.tensor` de shape `(batch_size, seq_len, vocab_size)` qui correspond aux vecteurs de probabilité de chaque token de la phrase actuellement prédite. `vocab_size` est le nombre de mots dans le vocabulaire du modèle. Donc pour obtenir le mot suivant, vous allez devoir trouver l'élément avec la plus grande probabilité dans le dernier vecteur de probabilité de `transformer(input_ids, decoder_input_ids)`.

`transformer.start_token` est l'id du token `<start>`. Vous allez devoir l'utiliser pour initialiser `decoder_input_ids` à la première étape de la boucle d'inférence.

In [ ]:
print(transformer.tokenize("I love oranges."))
print(transformer.detokenize([47, 1779, 12610, 9, 3, 0]))

Par exemple, si vous voulez prédire le premier mot de la phrase "I love oranges.", vous allez devoir faire:

In [ ]:
input_ids = transformer.tokenize("I love oranges.")
decoder_input_ids = torch.tensor([[transformer.start_token]], dtype=torch.long)

prediction = transformer(input_ids, decoder_input_ids)
print(prediction.shape)

id_next_word = torch.argmax(prediction[0, -1]).item()
print(id_next_word)

next_word = transformer.detokenize([id_next_word])
print(next_word)

### A vous de jouer
Maintenant que vous savez comment utiliser le modèle, vous allez devoir faire la boucle d'inférence pour traduire la phrase "I love oranges." en français en utilisant le modèle.

## Entraînement
Avant de regarder l'architecture d'un transfo, on va regarder comment on entraîne un transformer. La phase d'entraînement est un peu différente de la phase d'inférence.
En fait, pendant l'entraînement, au lieu de prédire un token à la fois, le transformer va prédire tous les tokens de la target sequence (la phrase qu'on doit prédire) en même temps.

Par exemple, reprenons l'exemple "I love oranges.". Notre target sequence est "J'aime les oranges.". Au lieu de prédire un mot à la fois, le transformer va prédire `[J', aime, les, oranges, ., <end>]`.

Plus précicément, on va donner au transformer les deux phrases `[I, love, oranges, .]` et `[\<start>, J', aime, les, oranges, .]` en `input_ids` et `decoder_input_ids` respectivement, et on veut qu'il:
    - prédise `J'` à partir de `[`<start>`]`
    - prédise `aime` à partir de `[`<start>`, J']`
    - prédise `les` à partir de `[`<start>`, J', aime]`
    - prédise `oranges` à partir de `[`<start>`, J', aime, les]`
    - prédise `.` à partir de `[`<start>`, J', aime, les, oranges]`
    - prédise `<end>` à partir de `[`<start>`, J', aime, les, oranges, .]`

Et le transformer va faire tout ça parallellement, en une seule passe.


![training phase](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/prediction.png)


